# Proceso de ETL:

## 1. Extracción

En esta etapa, se importan las bibliotecas necesarias y se cargan los conjuntos de datos desde archivos Excel. Los archivos fueron descargados desde la página [Buenos Aires Data](https://data.buenosaires.gob.ar/dataset/victimas-siniestros-viales), específicamente los recursos de lesiones y homicidios.


### Importación de Librerías
Se importan las bibliotecas necesarias para llevar a cabo el proceso ETL.

In [1]:
import pandas as pd
import numpy as np

### Lectura de Archivos:

En esta parte del proceso ETL, se lleva a cabo la lectura de los conjuntos de datos desde archivos Excel. Los archivos se ubican en la ruta './datasets/Originales' y contienen información sobre homicidios y lesiones relacionadas con accidentes viales en la ciudad de Buenos Aires.

In [2]:
homicidios_hechos= pd.read_excel('../datasets/Originales/homicidios.xlsx',sheet_name = 'HECHOS',na_values=['SD', 'sd'])
homicidios_victimas= pd.read_excel('../datasets/Originales/homicidios.xlsx',sheet_name = 'VICTIMAS',na_values=['SD', 'sd'])
lesiones_hechos= pd.read_excel('../datasets/Originales/lesiones.xlsx' , sheet_name = 'HECHOS',na_values=['SD', 'sd'])
lesiones_victimas= pd.read_excel('../datasets/Originales/lesiones.xlsx' , sheet_name = 'VICTIMAS',na_values=['SD', 'sd'])

## 2. Transformación:

En esta etapa, se realiza la manipulación y preparación de los datos para su posterior análisis. Las transformaciones que se llevarán a cabo son las siguientes:

- **Eliminación de columnas:** Se quitarán aquellas columnas que no se utilizarán en este proyecto.
- **Manejo de valores nulos:** Los valores 'SD' y 'sd' se han marcado como valores nulos durante la carga de datos.
- **Cambio de tipo de dato:** Se llevará a cabo la conversión de los datos al tipo más conveniente para el análisis.
- **Renombrar columnas:** Se realizará el cambio del nombre de las columnas según sea necesario.

### DataFrame homicidios_hechos

In [3]:
homicidios_hechos.head()

,ID,N_VICTIMAS,FECHA,AAAA,MM,DD,HORA,HH,LUGAR_DEL_HECHO,TIPO_DE_CALLE,...,Altura,Cruce,Dirección Normalizada,COMUNA,XY (CABA),pos x,pos y,PARTICIPANTES,VICTIMA,ACUSADO
0,2016-0001,1,2016-01-01,2016,1,1,04:00:00,4.0,AV PIEDRA BUENA Y AV FERNANDEZ DE LA CRUZ,AVENIDA,...,NaN,"FERNANDEZ DE LA CRUZ, F., GRAL. AV.","PIEDRA BUENA AV. y FERNANDEZ DE LA CRUZ, F., G...",8,Point (98896.78238426 93532.43437792),-58.47533969,-34.68757022,MOTO-AUTO,MOTO,AUTO
1,2016-0002,1,2016-01-02,2016,1,2,01:15:00,1.0,AV GRAL PAZ Y AV DE LOS CORRALES,GRAL PAZ,...,NaN,DE LOS CORRALES AV.,"PAZ, GRAL. AV. y DE LOS CORRALES AV.",9,Point (95832.05571093 95505.41641999),-58.50877521,-34.66977709,AUTO-PASAJEROS,AUTO,PASAJEROS
2,2016-0003,1,2016-01-03,2016,1,3,07:00:00,7.0,AV ENTRE RIOS 2034,AVENIDA,...,2034.0,NaN,ENTRE RIOS AV. 2034,1,Point (106684.29090040 99706.57687843),-58.39040293,-34.63189362,MOTO-AUTO,MOTO,AUTO
3,2016-0004,1,2016-01-10,2016,1,10,00:00:00,0.0,AV LARRAZABAL Y GRAL VILLEGAS CONRADO,AVENIDA,...,NaN,"VILLEGAS, CONRADO, GRAL.","LARRAZABAL AV. y VILLEGAS, CONRADO, GRAL.",8,Point (99840.65224780 94269.16534422),-58.46503904,-34.68092974,MOTO-SD,MOTO,NaN
4,2016-0005,1,2016-01-21,2016,1,21,05:20:00,5.0,AV SAN JUAN Y PRESIDENTE LUIS SAENZ PEÑA,AVENIDA,...,NaN,"SAENZ PE?A, LUIS, PRES.","SAN JUAN AV. y SAENZ PEÃ‘A, LUIS, PRES.",1,Point (106980.32827929 100752.16915795),-58.38718297,-34.62246630,MOTO-PASAJEROS,MOTO,PASAJEROS


#### Eliminación de columnas

La columna FECHA será conservada, mientras que las columnas AAAA, DD, y MM, que hacen referencia a información de la fecha, serán eliminadas.

In [4]:
homicidios_hechos.drop(columns=['AAAA','DD','MM'],inplace=True)

La columna HH, que representa la hora entera, será conservada, mientras que la columna HORA, que incluye los minutos, será eliminada.

In [5]:
homicidios_hechos.drop(columns=['HORA'],inplace=True)

Para la ubicación del hecho, se ha decidido utilizar las columnas `pos X` y `pos Y`, que hacen referencia a la latitud y longitud respectivamente. Por lo tanto, se procederá a eliminar las siguientes columnas:

- `LUGAR_DEL_HECHO`
- `Calle`
- `Altura`
- `Cruce`
- `Dirección Normalizada`
- `XY(CABA)`



In [6]:
homicidios_hechos.drop(columns=['LUGAR_DEL_HECHO','Calle','Altura','Cruce','Dirección Normalizada','XY (CABA)'],inplace=True)

#### Valores nulos

Calcular el porcentaje de valores nulos por columnas

In [7]:
# Imprimir el porcentaje de los valores nulos
print('Porcentaje valores nulos')
print('-'*24)
print((homicidios_hechos.isnull().sum()[homicidios_hechos.isnull().sum() > 0] / len(homicidios_hechos)) * 100)

Porcentaje valores nulos
------------------------
HH         0.143678
VICTIMA    1.293103
ACUSADO    3.304598
dtype: float64


El porcentaje de valores nulos en el conjunto de datos es bajo, afectando solo a 3 columnas. Sin embargo, se identificaron irregularidades en la columna de "Comuna", donde la presencia de "Comuna 0" indica una inconsistencia, ya que las comunas deben estar en el rango de 1 a 15. Estos casos también serán tratados como valores nulos. Además, se observaron puntos en las columnas "Pos X" y "Pos Y", y se decidirá reemplazarlos por valores nulos para mantener la consistencia en la representación de coordenadas geográficas.

In [8]:
# Reemplazo de la Comuna 0 por Valores Nulos
homicidios_hechos.loc[homicidios_hechos['COMUNA'] == 0, 'COMUNA'] = np.nan
#Reemplazo de los . por valores nulos de las columnas pos x y pos y

homicidios_hechos.loc[homicidios_hechos['pos x'] == '.' ,'pos x'] = np.nan
homicidios_hechos.loc[homicidios_hechos['pos y'] == '.' ,'pos y'] = np.nan

In [9]:
# Imprimir el porcentaje de los valores nulos
print('Porcentaje valores nulos')
print('-'*24)
print((homicidios_hechos.isnull().sum()[homicidios_hechos.isnull().sum() > 0] / len(homicidios_hechos)) * 100)

Porcentaje valores nulos
------------------------
HH         0.143678
COMUNA     0.287356
pos x      1.724138
pos y      1.724138
VICTIMA    1.293103
ACUSADO    3.304598
dtype: float64


### Cambio de tipos de datos

Las columnas 'HH', 'TIPO_DE_CALLE', 'COMUNA' y 'ACUSADO' serán transformadas al tipo de dato categórico.

In [10]:
# cambio a tipo categórico
homicidios_hechos['HH'] = homicidios_hechos['HH'].astype('category')
homicidios_hechos['TIPO_DE_CALLE'] = homicidios_hechos['TIPO_DE_CALLE'].astype('category')
homicidios_hechos['COMUNA'] = homicidios_hechos['COMUNA'].astype('category')
homicidios_hechos['ACUSADO'] = homicidios_hechos['ACUSADO'].astype('category')

Las columnas 'pos X' y 'pos Y' serán transformadas al tipo de dato flotante. 

In [11]:
# cambio a flotante los valores de pos x y pos y
homicidios_hechos['pos x'] = homicidios_hechos['pos x'].astype('float')
homicidios_hechos['pos y'] = homicidios_hechos['pos y'].astype('float')

### Renombrar columnas

Todos los nombres de las columnas serán convertidos a minúsculas. 

In [12]:
homicidios_hechos.columns = homicidios_hechos.columns.str.lower()

Renombrar las columnas de 'id', 'pos x' y 'pos y'

In [13]:
# Renombrar columnas de ID, Pos X y Pos Y
homicidios_hechos.rename(columns={'id': 'id_hecho', 'pos x': 'longitud', 'pos y': 'latitud','tipo_de_calle':'tipo_calle'}, inplace=True)

Las columnas 'tipo_calle', 'participantes', 'victima' y 'acusado' cambiaron el contenido de sus filas en minúsculas.  

In [14]:
# Cambiar el contenido de las filas a minúsculas
columnas_a_minusc = ['tipo_calle', 'participantes', 'victima', 'acusado']
homicidios_hechos[columnas_a_minusc] = homicidios_hechos[columnas_a_minusc].apply(lambda x: x.astype(str).str.lower())

Para facilitar el análisis en etapas posteriores del proyecto, hemos llevado a cabo la separación de la columna 'participantes' en dos nuevas columnas: 'vehiculo1' y 'vehiculo2'.

In [15]:
# Crear nuevas columnas 'vehiculo1' y 'vehiculo2' a partir de 'participantes'
homicidios_hechos[['vehiculo1', 'vehiculo2']] = homicidios_hechos['participantes'].str.split('-', expand=True)

# Reemplazar 'sd' con NaN en ambas columnas
homicidios_hechos[['vehiculo1', 'vehiculo2']] = homicidios_hechos[['vehiculo1', 'vehiculo2']].replace('sd',np.nan)

#Borrar la columna participantes
homicidios_hechos.drop('participantes',axis=1, inplace=True)


Conversión de los 'nan' a valores nulos

In [16]:
homicidios_hechos.replace('nan', np.nan, inplace=True)

Creamos la columna 'gravedad' y asignamos el valor 'fatal'

In [17]:
homicidios_hechos['gravedad']='fatal'

Cambio de nombre de las filas en las columna 'victima'

In [18]:
# Definir el diccionario de mapeo para las correcciones ortográficas
mapeo_correcciones_victima = {'peaton': 'peatón','movil': 'móvil'}

# Aplicar las correcciones a la columna 'victima'
homicidios_hechos['victima'] = homicidios_hechos['victima'].replace(mapeo_correcciones_victima)

In [19]:
homicidios_hechos.head()

,id_hecho,n_victimas,fecha,hh,tipo_calle,comuna,longitud,latitud,victima,acusado,vehiculo1,vehiculo2,gravedad
0,2016-0001,1,2016-01-01,4.0,avenida,8.0,-58.475340,-34.687570,moto,auto,moto,auto,fatal
1,2016-0002,1,2016-01-02,1.0,gral paz,9.0,-58.508775,-34.669777,auto,pasajeros,auto,pasajeros,fatal
2,2016-0003,1,2016-01-03,7.0,avenida,1.0,-58.390403,-34.631894,moto,auto,moto,auto,fatal
3,2016-0004,1,2016-01-10,0.0,avenida,8.0,-58.465039,-34.680930,moto,NaN,moto,NaN,fatal
4,2016-0005,1,2016-01-21,5.0,avenida,1.0,-58.387183,-34.622466,moto,pasajeros,moto,pasajeros,fatal


In [20]:
homicidios_hechos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 696 entries, 0 to 695
Data columns (total 13 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   id_hecho    696 non-null    object        
 1   n_victimas  696 non-null    int64         
 2   fecha       696 non-null    datetime64[ns]
 3   hh          695 non-null    category      
 4   tipo_calle  696 non-null    object        
 5   comuna      694 non-null    category      
 6   longitud    684 non-null    float64       
 7   latitud     684 non-null    float64       
 8   victima     687 non-null    object        
 9   acusado     673 non-null    object        
 10  vehiculo1   687 non-null    object        
 11  vehiculo2   656 non-null    object        
 12  gravedad    696 non-null    object        
dtypes: category(2), datetime64[ns](1), float64(2), int64(1), object(7)
memory usage: 62.7+ KB


### DataFrame homicidios_victimas

In [21]:
homicidios_victimas.head()

,ID_hecho,FECHA,AAAA,MM,DD,ROL,VICTIMA,SEXO,EDAD,FECHA_FALLECIMIENTO
0,2016-0001,2016-01-01,2016,1,1,CONDUCTOR,MOTO,MASCULINO,19.0,2016-01-01 00:00:00
1,2016-0002,2016-01-02,2016,1,2,CONDUCTOR,AUTO,MASCULINO,70.0,2016-01-02 00:00:00
2,2016-0003,2016-01-03,2016,1,3,CONDUCTOR,MOTO,MASCULINO,30.0,2016-01-03 00:00:00
3,2016-0004,2016-01-10,2016,1,10,CONDUCTOR,MOTO,MASCULINO,18.0,NaN
4,2016-0005,2016-01-21,2016,1,21,CONDUCTOR,MOTO,MASCULINO,29.0,2016-02-01 00:00:00


#### Eliminación de columnas

La columna FECHA será conservada, mientras que las columnas AAAA, DD, y MM, que hacen referencia a información de la fecha, serán eliminadas.

In [22]:
#Eliminación de columnas
homicidios_victimas.drop(columns=['AAAA','MM','DD','FECHA_FALLECIMIENTO'],inplace=True)

#### Valores nulos
Calcular el porcentaje de valores nulos por columnas

In [23]:
# Imprimir el porcentaje de los valores nulos
print('Porcentaje valores nulos')
print('-'*24)
print((homicidios_victimas.isnull().sum()[homicidios_victimas.isnull().sum() > 0] / len(homicidios_victimas)) * 100)

Porcentaje valores nulos
------------------------
ROL        1.534170
VICTIMA    1.255230
SEXO       0.836820
EDAD       7.391911
dtype: float64


### Cambio de tipos de datos

Conversión de las columnas 'ROL', 'VICTIMA' y 'SEXO' a tipo categórico

In [24]:
# cambio a formato categórico
homicidios_victimas['ROL'] = homicidios_victimas['ROL'].astype('category')
homicidios_victimas['VICTIMA'] = homicidios_victimas['VICTIMA'].astype('category')
homicidios_victimas['SEXO'] = homicidios_victimas['SEXO'].astype('category')


### Renombrar columnas

Todos los nombres de las columnas serán convertidos a minúsculas. 

In [25]:
homicidios_victimas.columns = homicidios_victimas.columns.str.lower()

Cambiamos el nombre de las columnas

In [26]:
homicidios_victimas.rename(columns={'edad': 'edad_victima','victima':'vehiculo_victima','sexo':'sexo_victima'}, inplace=True)

A las columnas 'rol', 'victima', 'sexo' y 'acusado' se les cambió el contenido de sus filas a minúsculas.

In [27]:
# Cambiar el contenido de las filas a minúsculas en columnas específicas
columnas_a_minusc = ['rol', 'vehiculo_victima', 'sexo_victima']
homicidios_victimas[columnas_a_minusc] = homicidios_victimas[columnas_a_minusc].apply(lambda x: x.astype(str).str.lower())

Cambio de la etiqueta 'pasajero_acompañante' por 'pasajero' en la columna rol.

In [28]:
#Cambio de etiqueta
homicidios_victimas['rol'].replace('pasajero_acompañante', 'pasajero', inplace=True)

Cambiamos 'peaton' por 'peatón' en el rol

In [29]:
homicidios_victimas['rol'].replace('peaton', 'peatón', inplace=True)

Cambiamos los nombres de la columa 'vehiculo_victima'

In [30]:
# Diccionario de correcciones
diccionario_correcciones = {'movil': 'móvil', 'peaton': 'peatón'}

# Aplicar correcciones
homicidios_victimas['vehiculo_victima'] = homicidios_victimas['vehiculo_victima'].replace(diccionario_correcciones)

Creamos la columna 'gravedad' y asignamos el valor 'fatal'

In [31]:
homicidios_victimas['gravedad'] = 'fatal'

In [32]:
homicidios_victimas.head()

,id_hecho,fecha,rol,vehiculo_victima,sexo_victima,edad_victima,gravedad
0,2016-0001,2016-01-01,conductor,moto,masculino,19.0,fatal
1,2016-0002,2016-01-02,conductor,auto,masculino,70.0,fatal
2,2016-0003,2016-01-03,conductor,moto,masculino,30.0,fatal
3,2016-0004,2016-01-10,conductor,moto,masculino,18.0,fatal
4,2016-0005,2016-01-21,conductor,moto,masculino,29.0,fatal


### DataFrame lesiones_hechos

In [33]:
lesiones_hechos.head()

,id,n_victimas,aaaa,mm,dd,fecha,hora,franja_hora,direccion_normalizada,comuna,...,latutid,victima,acusado,participantes,moto,auto,transporte_publico,camion,ciclista,gravedad
0,LC-2019-0000179,1,2019,1,1,2019-01-01 00:00:00,09:00:00,9.0,NaN,14,...,-34.559658,CICLISTA,NaN,CICLISTA-SD,NaN,NaN,NaN,NaN,x,NaN
1,LC-2019-0000053,1,2019,1,1,2019-01-01 00:00:00,01:55:00,1.0,NaN,8,...,-34.669125,AUTO,NaN,AUTO-SD,NaN,x,NaN,NaN,NaN,NaN
2,LC-2019-0000063,1,2019,1,1,2019-01-01 00:00:00,02:00:00,2.0,NaN,8,...,-34.677556,NaN,NaN,SD-SD,NaN,NaN,NaN,NaN,NaN,NaN
3,LC-2019-0000079,1,2019,1,1,2019-01-01 00:00:00,02:30:00,2.0,NaN,7,...,-34.647349,PEATON,NaN,PEATON-SD,x,NaN,NaN,NaN,NaN,NaN
4,LC-2019-0000082,4,2019,1,1,2019-01-01 00:00:00,04:30:00,4.0,NaN,3,...,-34.604579,AUTO,NaN,AUTO-SD,NaN,NaN,x,NaN,NaN,NaN


#### Eliminación de columnas
La columna 'fecha' será conservada, mientras que las columnas 'aaaa', 'dd', y 'mm', que hacen referencia a información de la fecha, serán eliminadas.

In [34]:
lesiones_hechos.drop(columns=['aaaa','mm','dd'],inplace=True)

La columna 'franja_hora', que representa la hora entera, será conservada, mientras que la columna 'hora', que incluye los minutos, será eliminada.

In [35]:
lesiones_hechos.drop(columns=['hora'],inplace=True)

Para la ubicación del hecho, se ha decidido utilizar las columnas `latitud` y `longitud`. Por lo tanto, se procederá a eliminar las siguientes columnas:

- `otra_direccion`
- `calle`
- `altura`
- `cruce`
- `geocodificacion_CABA`
- `direccion_normalizada`

In [36]:
lesiones_hechos.drop(columns=['otra_direccion','calle','altura','cruce','geocodificacion_CABA','direccion_normalizada'],inplace=True)

En la columna 'participantes', se detalla la información sobre los involucrados en el incidente, lo cual hace que las columnas específicas para detallar la presencia de moto, auto, transporte público, camión y ciclista resulten redundantes. 

In [37]:
lesiones_hechos.drop(columns=['moto','auto','transporte_publico','camion','ciclista'],inplace=True)

#### Valores nulos
Calcular el porcentaje de valores nulos por columnas


In [38]:
# Imprimir el porcentaje de los valores nulos
print('Porcentaje valores nulos')
print('-'*24)
print((lesiones_hechos.isnull().sum()[lesiones_hechos.isnull().sum() > 0] / len(lesiones_hechos)) * 100)

Porcentaje valores nulos
------------------------
franja_hora     0.037839
comuna          4.267395
tipo_calle     46.436830
longitud        6.184570
latutid         6.184570
victima        45.125079
acusado        64.275804
gravedad       97.284003
dtype: float64


Los valores nulos presentes en la columna 'gravedad' serán sustituidos por 'leves', conforme a las indicaciones proporcionadas en el diccionario de datos, donde se establece que los valores nulos deben ser interpretados como incidentes de gravedad leve

In [39]:
lesiones_hechos['gravedad'].fillna('leve', inplace=True)

En la columna 'comuna', los datos etiquetados como 'No Especificada' serán convertidos a valores nulos. 

In [40]:
lesiones_hechos['comuna'].replace('No Especificada',np.nan,inplace=True)

### Cambio de tipos de datos
Las columnas 'franja_hora', 'tipo_calle', 'comuna' y 'acusado' serán transformadas al tipo de dato categórico.

In [41]:
# cambio a tipo categórico
lesiones_hechos['franja_hora'] = lesiones_hechos['franja_hora'].astype('category')
lesiones_hechos['tipo_calle'] = lesiones_hechos['tipo_calle'].astype('category')
lesiones_hechos['comuna'] = lesiones_hechos['comuna'].astype('category')
lesiones_hechos['acusado'] = lesiones_hechos['acusado'].astype('category')

Las columnas de 'longitud' y 'latitud', serán convertidas al tipo de dato flotante. 

In [42]:
# cambio a flotante los valores de pos x y pos y
lesiones_hechos['longitud'] = lesiones_hechos['longitud'].astype('float')
lesiones_hechos['latutid'] = lesiones_hechos['latutid'].astype('float')

La columna 'fecha' será pasada a formato fecha

In [43]:
# Convierte la columna 'fecha' a formato datetime
lesiones_hechos['fecha'] = pd.to_datetime(lesiones_hechos['fecha'], errors='coerce')

### Renombrar columnas
Renombrar las columnas de 'lesiones_hechos' para que concuerden con los nombres de las columnas de 'homicidios_hechos'.

In [44]:
# Renombrar columnas en lesiones_hechos
lesiones_hechos.rename(columns={'id': 'id_hecho', 'franja_hora': 'hh', 'latutid': 'latitud'}, inplace=True)

Para facilitar el análisis en etapas posteriores del proyecto, hemos llevado a cabo la separación de la columna 'participantes' en dos nuevas columnas: 'vehiculo1' y 'vehiculo2'.

In [45]:
# Crear nuevas columnas 'vehiculo1' y 'vehiculo2' a partir de 'participantes'
lesiones_hechos[['vehiculo1', 'vehiculo2']] = lesiones_hechos['participantes'].str.split('-', expand=True)

# Reemplazar 'sd' con NaN en ambas columnas
lesiones_hechos[['vehiculo1', 'vehiculo2']] = lesiones_hechos[['vehiculo1', 'vehiculo2']].replace('SD',np.nan)

#Borrar la columna participantes
lesiones_hechos.drop('participantes',axis=1, inplace=True)

Las columnas 'tipo_calle', 'gravedad', 'victima' , 'acusado',vehiculo1 y vehiculo2 cambiaron el contenido de sus filas a minúsculas. Y se hacen cambios de nombres para coincidir con 'homicidios_hechos'.

In [46]:
# Transformar a minúsculas las columnas en lesiones_hechos
columnas_a_minusc = ['tipo_calle', 'gravedad', 'victima', 'acusado', 'vehiculo1', 'vehiculo2']
lesiones_hechos[columnas_a_minusc] = lesiones_hechos[columnas_a_minusc].apply(lambda x: x.astype(str).str.lower())

# Definir el diccionario de mapeo para las correcciones ortográficas en la columna 'victima'
mapeo_correcciones_victima_lesiones = {'ciclista': 'bicicleta','peaton': 'peatón','transporte publico': 'transporte público','movil': 'móvil','otro': 'otro',
'camion': 'camión','monopatin': 'monopatín','utilitario': 'utilitario',

}

# Aplicar las correcciones a la columna 'victima' en 'lesiones_hechos'
lesiones_hechos['victima'] = lesiones_hechos['victima'].replace(mapeo_correcciones_victima_lesiones)


Convertimos los 'nan' en valores nulos

In [47]:
lesiones_hechos.replace('nan', np.nan, inplace=True)

In [48]:
lesiones_hechos.head()

,id_hecho,n_victimas,fecha,hh,comuna,tipo_calle,longitud,latitud,victima,acusado,gravedad,vehiculo1,vehiculo2
0,LC-2019-0000179,1,2019-01-01,9.0,14.0,NaN,-58.408911,-34.559658,bicicleta,NaN,leve,ciclista,NaN
1,LC-2019-0000053,1,2019-01-01,1.0,8.0,NaN,-58.443510,-34.669125,auto,NaN,leve,auto,NaN
2,LC-2019-0000063,1,2019-01-01,2.0,8.0,NaN,-58.468335,-34.677556,NaN,NaN,leve,NaN,NaN
3,LC-2019-0000079,1,2019-01-01,2.0,7.0,NaN,-58.437425,-34.647349,peatón,NaN,leve,peaton,NaN
4,LC-2019-0000082,4,2019-01-01,4.0,3.0,NaN,-58.398225,-34.604579,auto,NaN,leve,auto,NaN


### DataFrame lesiones_victimas

In [49]:
lesiones_victimas.head()

,ID hecho,AAA,MM,DD,FECHA,VEHICULO_VICTIMA,SEXO,EDAD_VICTIMA,GRAVEDAD
0,LC-2019-0000053,2019,1,1,2019-01-01,NaN,Varon,57.0,NaN
1,LC-2019-0000063,2019,1,1,2019-01-01,NaN,NaN,NaN,NaN
2,LC-2019-0000079,2019,1,1,2019-01-01,NaN,Varon,NaN,NaN
3,LC-2019-0000082,2019,1,1,2019-01-01,NaN,Varon,45.0,NaN
4,LC-2019-0000082,2019,1,1,2019-01-01,NaN,Mujer,45.0,NaN


La columna 'FECHA' será conservada, mientras que las columnas 'AAA', 'DD', y 'MM', que hacen referencia a información de la fecha, serán eliminadas.

In [50]:
#Eliminación de columnas
lesiones_victimas.drop(columns=['AAA','MM','DD'],inplace=True)

#### Valores nulos
Calcular el porcentaje de valores nulos por columnas

In [51]:
# Imprimir el porcentaje de los valores nulos
print('Porcentaje valores nulos')
print('-'*24)
print((lesiones_victimas.isnull().sum()[lesiones_victimas.isnull().sum() > 0] / len(lesiones_victimas)) * 100)

Porcentaje valores nulos
------------------------
VEHICULO_VICTIMA    58.945843
SEXO                 8.071002
EDAD_VICTIMA        16.196341
GRAVEDAD            98.054700
dtype: float64


Los valores nulos presentes en la columna 'GRAVEDAD' serán sustituidos por 'leves', conforme a las indicaciones proporcionadas en el diccionario de datos, donde se establece que los valores nulos deben ser interpretados como incidentes de gravedad leve

In [52]:
lesiones_victimas['GRAVEDAD'].fillna('leve', inplace=True)

### Cambio de tipos de datos

Conversión de las columnas 'VEHICULO_VICTIMA', 'GRAVEDAD' y 'SEXO' a tipo categórico

In [53]:
# cambio a tipo categórico
lesiones_victimas['VEHICULO_VICTIMA'] = lesiones_victimas['VEHICULO_VICTIMA'].astype('category')
lesiones_victimas['GRAVEDAD'] = lesiones_victimas['GRAVEDAD'].astype('category')
lesiones_victimas['SEXO'] = lesiones_victimas['SEXO'].astype('category')

### Renombrar columnas
Renombrar las columnas de lesiones_victimas para que concuerden con los nombres de las columnas de homicidios_victimas.

In [54]:
# Transformar a minúsculas todas las columnas en lesiones_victimas
lesiones_victimas.columns = lesiones_victimas.columns.str.lower()
# Renombrar columnas en lesiones_victimas
lesiones_victimas.rename(columns={'id hecho': 'id_hecho', 'fecha ': 'fecha', 'sexo': 'sexo_victima'}, inplace=True)


Las columnas 'vehiculo_victima', 'sexo_victima' y 'gravedad' se han transformado a minúsculas 

In [55]:
lesiones_victimas[['vehiculo_victima', 'sexo_victima', 'gravedad']] = lesiones_victimas[['vehiculo_victima', 'sexo_victima', 'gravedad']].apply(lambda x: x.astype(str).str.lower())

La columna 'sexo_victima', los valores 'varon' y 'mujer' serán cambiados por 'masculino' y 'femenino'.

In [56]:
lesiones_victimas['sexo_victima'].replace({'varon': 'masculino', 'mujer': 'femenino','mujer ': 'femenino'}, inplace=True)

Corregimos la ortografía de la columna 'vehiculo_victima'.

In [57]:
# Diccionario
diccionario_correcciones = {'movil': 'móvil','camion': 'camión','monopatin': 'monopatín','peaton':'peatón','ciclista':'bicicleta',
                            'transporte publico':'transporte público'}
#Correción
lesiones_victimas['vehiculo_victima'] = lesiones_victimas['vehiculo_victima'].replace(diccionario_correcciones.keys(), diccionario_correcciones.values(), regex=True)

Convertimos los valores 'nan' en valores nulos

In [58]:
lesiones_victimas.replace('nan', np.nan, inplace=True)

In [59]:
lesiones_victimas.head()

,id_hecho,fecha,vehiculo_victima,sexo_victima,edad_victima,gravedad
0,LC-2019-0000053,2019-01-01,NaN,masculino,57.0,leve
1,LC-2019-0000063,2019-01-01,NaN,NaN,NaN,leve
2,LC-2019-0000079,2019-01-01,NaN,masculino,NaN,leve
3,LC-2019-0000082,2019-01-01,NaN,masculino,45.0,leve
4,LC-2019-0000082,2019-01-01,NaN,femenino,45.0,leve


### Concatenación de los DataFrames

Realizaremos la concatenación de los DataFrames 'homicidios_hechos' y 'lesiones_hechos', así como de los DataFrames 'homicidios_victimas' y 'lesiones_victimas.

In [60]:
# Define el nuevo orden de las columnas para 'homicidios_hechos'
nuevo_orden_homicidios_hechos = ['id_hecho', 'n_victimas', 'fecha', 'hh', 'tipo_calle', 'comuna',
                                'longitud', 'latitud', 'victima', 'acusado', 'vehiculo1', 'vehiculo2',
                                'gravedad']

# Reordena las columnas en 'homicidios_hechos' según el nuevo orden
homicidios_hechos = homicidios_hechos[nuevo_orden_homicidios_hechos]

# Define el nuevo orden de las columnas para 'lesiones_hechos'
nuevo_orden_lesiones_hechos = ['id_hecho', 'n_victimas', 'fecha', 'hh', 'comuna', 'tipo_calle',
                               'longitud', 'latitud', 'victima', 'acusado', 'gravedad', 'vehiculo1',
                               'vehiculo2']

# Reordena las columnas en 'lesiones_hechos' según el nuevo orden
lesiones_hechos = lesiones_hechos[nuevo_orden_lesiones_hechos]

# Concatena los DataFrames
hechos = pd.concat([homicidios_hechos, lesiones_hechos], ignore_index=True)



Modificamos la autopista 'gral paz' por autopista

In [61]:
hechos['tipo_calle'] = hechos['tipo_calle'].replace('gral paz', 'autopista')

Concatenamos los DataFrames de las víctimas

In [62]:
# Define el nuevo orden de las columnas para 'homicidios_victimas'
nuevo_orden_homicidios_victimas = ['id_hecho', 'fecha', 'rol', 'vehiculo_victima', 'sexo_victima',
                                   'edad_victima', 'gravedad']

# Reordena las columnas en 'homicidios_victimas' según el nuevo orden
homicidios_victimas = homicidios_victimas[nuevo_orden_homicidios_victimas]

# Define el nuevo orden de las columnas para 'lesiones_victimas'
nuevo_orden_lesiones_victimas = ['id_hecho', 'fecha', 'vehiculo_victima', 'sexo_victima', 'edad_victima',
                                 'gravedad']

# Reordena las columnas en 'lesiones_victimas' según el nuevo orden
lesiones_victimas = lesiones_victimas[nuevo_orden_lesiones_victimas]

# Concatena los DataFrames
victimas = pd.concat([homicidios_victimas, lesiones_victimas], ignore_index=True)

Realizar la modificación en el tipo de datos de nuestro DataFrame 'victimas' 

In [63]:
# Diccionario de conversión
conversiones_victimas = {'id_hecho': str,'rol': 'category','vehiculo_victima': 'category','sexo_victima': 'category','gravedad': 'category'
}

# Aplicar las conversiones 
victimas = victimas.astype(conversiones_victimas)

## 3. Carga:

Convertir los DataFrames 'victimas' y 'hechos' en archivos CSV.

In [64]:
# Guardar el DataFrame 'hechos' en un archivo CSV
hechos.to_csv('../Datasets/CSV/hechos.csv', index=False)

# Guardar el DataFrame 'victimas' en un archivo CSV
victimas.to_csv('../Datasets/CSV/victimas.csv', index=False)